## Full Chatbot Conversation Notebook

This notebook is meant to be used to run the whole pipeline for using the chatbot. 
<br> <br>
Insert prompts/ values for `PERSONA`, `CLASSIFICATION_PROMPT`, and `TEMPERATURE` from the `prompt_classification.ipynb` notebook below.  

* Note: `CLASSIFICATION_PROMPT` is just a string combining it with `DATA_REQ` now, choose the best prompt from the previously notebook to put here.

In [ ]:
PERSONA: str = '''You are tasked with answering user queries for a hotel booking website. Your name is Nexus.'''

CLASSIFICATION_PROMPT: str = '''Given the users prompt below classify what they are talking about into the following 4 categories: 
account query, hotel recommendation query, conversing, misc.
If they are asking about something related to their account respond with:
{ "type": "ACC_QUERY" }

If asking about adding a hotel to their watchlist, booking, or getting recommendations respond with. 
If the user does not provide enough information for you to fill out all of the data fields, then fill then in with "NULL". 
Make sure you follow JSON format so things are in quotes especially "NULL":
{ 
    "type": "HOTEL_RECC",
    "data": {
        "location": str,
        "check_in": str,
        "check_out": str,
        "rooms": int,
        "adults": int,
        "children": int,
        "radius": int,
        "min_price": int,
        "max_price": int,
    }
}

If they are continuing a conversation with you regarding booking a hotel, respond with. Update the data with what you knew previously
and what new information they give:
{ 
    "type": "CONVERSING"
    "data": {
        "location": str,
        "check_in": str,
        "check_out": str,
        "rooms": int,
        "adults": int,
        "children": int,
        "radius": int,
        "min_price": int,
        "max_price": int,
    }
}

If they are asking about something unrelated to the above politely generate a fitting response. For example if they are asking about
something not related to their LikeHome account then say you cannot help with that. If they are thanking you for helping them
then respond accordingly.

Answer with these instructions strictly and do not waver from it.
'''

TEMPERATURE: float = 0.2

### Conversation Prompts

Fill in several of the below prompts to test them simulatenously. Make sure lists are of the same length.

In [ ]:
ACCOUNT_CONVERSE_PROMPT: list = [
    ''' Given the following user account details: 
        {"name": "Bob", "email": "BobJones@email.com"}
    return to the user what they want.''',
    
    ''' Given the following user account details: 
        {"name": "Bob", "email": "BobJones@email.com"}
    polite create a uniqur response according to what the user wants. '''
]

HOTEL_CONVERSE_PROMPT: list = [
    '''If you have all the fields filled out generate a message thanking the user and saying a hotel has successfully been added to their 
    watchlist. If you do not have all the fields, ask the user politely for the remaining fields you need.''',
    
    '''If you have all the fields filled out generate a message thanking the user and saying a hotel has successfully been added to their 
    watchlist. If you do not have all the fields, ask the user politely for the remaining fields you need, generating a unique message. 
    If conversation goes to long, say 3-4 messages from the user then prefill the prompts with your best shot to prevent angression.'''
]

### Set-Up

Run the below cells to set up the function calls

In [ ]:
from openai import OpenAI
import numpy as np
import pandas as pd
from IPython.display import display
import json

client = OpenAI()
conversations = []
# initialize all conversations
for i in ACCOUNT_CONVERSE_PROMPT:
    conversations.append([{"role": "system", "content": PERSONA}])

# client to request gpt
def generate_response(messages: str):
    completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=TEMPERATURE,
        )
    response = completion.choices[0].message.content
    # TODO see if it is good to append this formatted response
    messages.append({"role": "assistant", "content": response})

    return response

# first step in pipeline to classify
def classify(prompt: str, conv: list):
    conv.append({"role": "system", "content": CLASSIFICATION_PROMPT})
    conv.append({"role": "user", "content": prompt})
    response = generate_response(conv)

    return response

# second step in conversing or completing
def converse(obj: dict, conv: list, hotel_conv: str, acc_conv: str):
    class_type = obj['type']

    if class_type == "HOTEL_RECC" or class_type == "CONVERSING":
        conv.append({"role": "system", "content": hotel_conv})
        return generate_response(conv)
    elif class_type == "ACC_QUERY":
        conv.append({"role": "system", "content": acc_conv})
        return generate_response(conv)

    # should not reach here
    print(obj)
    return "error"

def create_df(prompt1: list, prompt2: list, response1: str, response2: str):
    df = pd.DataFrame({
        "Hotel Converse Prompt": prompt1,
        "Account Converse Prompt": prompt2,
        "Class Response": response1,
        "User Receive Response": response2,
    })
    df_styled = df.style.set_table_styles([
        {
            'selector': 'table',
            'props': [('width', '100%'), ('overflow', 'hidden')]  
        },
        {
            'selector': 'td',  
            'props': [
                ('max-height', '100px'),
                ('max-width', '500px'),
                ('overflow', 'auto'), 
                ('white-space', 'pre'), 
                ('text-align', 'left'), 
                ('padding', '5px'),     
                ('border', '1px solid black') 
            ]
        }
    ])
    return df_styled

# entry point function for chatbot
def chat(prompt: str):
    global conversations
    # store responses for output df
    class_response = []
    final_response = []
    
    for i in range(len(HOTEL_CONVERSE_PROMPT)):
        prompt_class = classify(prompt, conversations[i])
        class_response.append(prompt_class)
        try:
            class_obj = json.loads(prompt_class)
            final_response.append(converse(class_obj, conversations[i], HOTEL_CONVERSE_PROMPT[i], ACCOUNT_CONVERSE_PROMPT[i]))
            
        except Exception as e: # in case that class returned non obj
            print(e)
            final_response.append(prompt_class)

    display(create_df(HOTEL_CONVERSE_PROMPT, ACCOUNT_CONVERSE_PROMPT, class_response, final_response))
    ''' print messages in conversation
    for conversation in conversations:
        print("conversation start:")
        for msg in conversation:
            print(msg)
            print("")
        print("")
    '''

### Run 

Run the below cell to start the chatbot. Type `quit` to end the conversation.

In [ ]:
def main():
    while(True):
        prompt = input("chat with Nexus: ")
        if prompt == "quit":
            print("chat ended")
            break
        chat(prompt)

main()